In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('./data/mobile.bg-offers-2024-02-17.csv')

In [ ]:
data

In [ ]:
# Translate column names to English for easier use
english_columns = ['Make', 'Model', 'BodyType', 'ManufactureDate', 'FuelType', 'Transmission', 'EngineSize', 'Horsepower', 'EuroStandard', 'Mileage', 'MilesPerFullCharge (EV)', 'BatteryCapacity (EV)', 'Color', 'VIN', 'Price', 'SafetyFeatures', 'ComfortFeatures', 'OtherFeatures', 'ExteriorFeatures', 'SecurityFeatures', 'InteriorFeatures', 'SpecialisedFeatures', 'Region', 'City', 'ViewCount', 'OfferTitle']

data.columns = english_columns
data

In [ ]:
data.shape

In [ ]:
data.dtypes

In [ ]:
numeric_columns = data.select_dtypes(include=[np.number]).columns
categorical_columns = data.select_dtypes(include=[object]).columns

numeric_columns, categorical_columns

In [ ]:
# Extract month from the Year of manufacture column

extracted_months = data['ManufactureDate'].str.split().str[0]
extracted_years = data['ManufactureDate'].str.split().str[1]

month_index_dict = {
    'януари': '01',
    'февруари': '02',
    'март': '03',
    'април': '04',
    'май': '05',
    'юни': '06',
    'юли': '07',
    'август': '08',
    'септември': '09',
    'октомври': '10',
    'ноември': '11',
    'декември': '12',
}

extracted_months = extracted_months.map(lambda x: month_index_dict.get(x, np.nan))

data['ManufactureDate'] = extracted_months.str.cat(extracted_years, sep='/')

In [ ]:
data['ManufactureDate'] = data['ManufactureDate'].str.replace('г.', '')

data['ManufactureDate']

In [ ]:
data['ManufactureDate'].isna().sum()

In [ ]:
data.head(20)

In [ ]:
data['FuelType'].value_counts(dropna=False)

In [ ]:
data['Transmission'].value_counts(dropna=False)

In [ ]:
data['EngineSize'].sample(20)

In [ ]:
# Ensure 'EngineSize' is a string, replace ' куб.см', and convert to integer
data.loc[data['EngineSize'].notna(), 'EngineSize'] = data.loc[data['EngineSize'].notna(), 'EngineSize'].str.replace(' куб.см', '').astype('Int64')

In [ ]:
data['EngineSize'].sample(20)

In [ ]:
data['EngineSize'].value_counts(dropna=False)

In [ ]:
data.head()

In [ ]:
data['Horsepower'].sample(20)

In [ ]:
data['Horsepower'].value_counts(dropna=False)

In [ ]:
data['Horsepower'] = data['Horsepower'].str.replace(' к.с.', '').astype('Int64')

data['Horsepower'].value_counts(dropna=False)

In [ ]:
data.head()

In [ ]:
data['EuroStandard'].value_counts(dropna=False)

In [ ]:
data['Mileage'].sample(20)

In [ ]:
data['Mileage'].value_counts(dropna=False)

In [ ]:
data['Mileage'] = data['Mileage'].str.replace(' км', '').astype('Int64')

data['Mileage'].value_counts(dropna=False)

In [ ]:
data.head()

In [ ]:
numeric_columns = data.select_dtypes(include=[np.number]).columns

numeric_columns

In [ ]:
data['MilesPerFullCharge (EV)'].value_counts(dropna=False)

In [ ]:
data[data['MilesPerFullCharge (EV)'].notna()].sample(20)

In [ ]:
data['MilesPerFullCharge (EV)'] = data['MilesPerFullCharge (EV)'].str.replace(' км', '').astype('Float64')

data['MilesPerFullCharge (EV)'].value_counts(dropna=False)

In [ ]:
numeric_columns = data.select_dtypes(include=[np.number]).columns
numeric_columns

In [ ]:
data['BatteryCapacity (EV)'].value_counts(dropna=False)

In [ ]:
data['BatteryCapacity (EV)'] = data['BatteryCapacity (EV)'].str.replace(' kWh', '').astype('Float64')

In [ ]:
data['BatteryCapacity (EV)'].value_counts(dropna=False)

In [ ]:
data['Color'].value_counts(dropna=False)

In [ ]:
data['VIN'].value_counts(dropna=False)

In [ ]:
data.head()

In [ ]:
data['Price'].sample(20)

In [ ]:
data['Price'].value_counts()

In [ ]:

price_in_eur = data['Price'].str.contains('EUR')
valid_price_rows = ~data['Price'].str.contains('При запитване')

data.loc[valid_price_rows, 'Price'] = data.loc[valid_price_rows, 'Price'].str.replace(' лв.', '').str.replace(' EUR', '').str.replace(' ', '')

# Convert EUR prices to BGN
eur_bgn_exchange_rate = 1.95583
data.loc[price_in_eur, 'Price'] = (data.loc[price_in_eur, 'Price'].astype('float') * eur_bgn_exchange_rate).astype(object)

data['Price'].sample(20)

In [ ]:
data['Price'].value_counts(dropna=False)

In [ ]:
data['Price'].sample(20)

In [ ]:
data.head()

In [ ]:
numeric_columns = data.select_dtypes(include=[np.number]).columns
categorical_columns = data.select_dtypes(include=[object]).columns

numeric_columns, categorical_columns

In [ ]:
data['Region'].value_counts(dropna=False)

In [ ]:
data['City'].value_counts(dropna=False)

In [ ]:
numeric_columns = data.select_dtypes(include=[np.number]).columns
categorical_columns = data.select_dtypes(include=[object]).columns

numeric_columns, categorical_columns

In [ ]:
data['BodyType'].value_counts(dropna=False)

In [ ]:
data.tail()

In [ ]:
# Handle duplicated rows

# Exclude the columns which are not relevant for the comparison
# Some offers are listed 2 TIMES - under grouped models (like BMW 3 Series, 5 Series, etc.) AND also under the specific model in the group (like BMW 320, 520, etc.)

# Let's check by VIN number for example as it should be unique for each vehicle
duplicate_vin_rows = data[data.duplicated(subset=['VIN'], keep=False)].dropna(subset=['VIN'])

duplicate_vin_rows.sort_values(by='VIN')

In [ ]:
duplicate_vin_rows['Make'].unique()

In [ ]:
duplicated_columns = data.columns.difference(['ViewCount', 'OfferTitle', 'Model'])
duplicated_columns

In [ ]:
all_duplicates = data.duplicated(subset=duplicated_columns, keep=False)

data[all_duplicates].sort_values(by='Model')

In [ ]:
before_drop_rows = data.shape[0]
print(f'Before removing duplicates: {before_drop_rows}')

data = data.drop_duplicates(subset=duplicated_columns, keep='last')
nan_rows = data.loc[data['VIN'].isna()]
data = data.drop_duplicates(subset=['VIN'], keep='last')
data = data.merge(nan_rows, how='outer')

print(f'{before_drop_rows - data.shape[0]} duplicates removed')

In [ ]:
data[data.duplicated(subset=duplicated_columns, keep=False)], data[data.duplicated(subset='VIN', keep=False)]

In [ ]:
data[data.duplicated()]

In [ ]:
print(f'Final shape: {data.shape}')
print(f'Final columns: {data.columns}')

In [ ]:
data.columns

In [ ]:
print(f'Final dtypes:')
data.dtypes

In [ ]:
data.to_csv('./data/bg-car-offers.csv', index=False)